In [ ]:
# LangChain官方文档教程
# https://python.langchain.com/docs/tutorials/agents/


In [ ]:
# Build an Agent（构建代理）
# LangChain 支持创建代理 ，即使用 LLM 作为推理引擎的系统，以确定要采取的操作以及执行操作所需的输入。执行操作后，结果可以反馈给 LLM，以确定是否需要执行更多操作，或者是否可以完成。这通常通过工具调用来实现。
# 在本教程中，我们将构建一个可以与搜索引擎交互的代理。您将能够向该代理提问，观察它如何调用搜索工具，并与其进行对话。


In [ ]:
# End-to-end agent  端到端代理
# 下面的代码片段代表一个功能齐全的代理，它使用 LLM 来决定使用哪些工具。它配备了通用搜索工具。它具有对话记忆功能，这意味着它可以用作多轮聊天机器人。
# 在本指南的其余部分，我们将介绍各个组件以及每个部分的功能 - 但如果您只想获取一些代码并开始使用，请随意使用它！



In [ ]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = init_chat_model("anthropic:claude-3-5-sonnet-latest")
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Bob and I live in SF.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

In [ ]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

In [ ]:
# Setup  设置
# Jupyter Notebook
# 本指南（以及文档中的大多数其他指南）使用 Jupyter Notebook ，并假设读者也使用 Jupyter Notebook。Jupyter Notebook 是学习如何使用 LLM 系统的理想交互式环境，因为很多情况下可能会出错（例如意外输出、API 故障等），而观察这些情况是更好地理解使用 LLM 进行构建的好方法。


In [ ]:
# Installation  安装
# pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

In [ ]:
# LangSmith
# 使用 LangChain 构建的许多应用程序都包含多个步骤，需要多次调用 LLM 函数。随着这些应用程序变得越来越复杂，能够检查链或代理内部究竟发生了什么变得至关重要。最好的方法是使用 LangSmith 。

# 通过上面的链接注册后，请确保设置环境变量以开始记录跟踪：

# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="..."

# 或者，如果在笔记本中，您可以使用以下方式设置它们：

# import getpass
# import os

# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()




In [1]:
# Tavily
# 我们将使用 Tavily （一个搜索引擎）作为工具。为了使用它，您需要获取并设置一个 API 密钥：

# export TAVILY_API_KEY="..."


# 或者，如果在笔记本中，您可以使用以下方式进行设置：

# import getpass
# import os

# os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [1]:
import getpass
import os
if not os.environ.get("TAVILY_API_KEY"):
  os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [2]:
# Define tools  定义工具

# 首先，我们需要创建想要使用的工具。我们主要选择的工具是 Tavily—— 一个搜索引擎。我们可以使用专用的 langchain-tavily 集成包 ，轻松地将 Tavily 搜索引擎作为 LangChain 的工具使用。



In [3]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("今天北京的天气如何？")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': '今天北京的天气如何？', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.weather.com.cn/weather/101010100.shtml', 'title': '预报- 北京 - 中国天气网', 'content': '<<返回 全国 全国>北京>城区 31/18℃ <3级 晴转多云 33/20℃ <3级 29/18℃ <3级 26/17℃ <3级 23/16℃ <3级 26/18℃ <3级 27/19℃ <3级 适宜 洗车指数 适宜 洗车指数 较适宜 洗车指数 适宜 洗车指数 适宜 洗车指数 适宜 洗车指数 过去24小时最大风力: 3级 (h)℃(μg/m³)(%)(mm)(级) 40°C 35°C 30°C 25°C 20°C 15°C 10°C 31  18℃  33  20℃  29  18℃  26  17℃  23  16℃  26  18℃  27  19℃  周边地区 | 周边景点 2025-05-19 07:30更新 周边地区 | 周边景点 2025-05-19 07:30更新 5月19日 今年首场大范围高温进入鼎盛阶段 南方降雨陷入“车轮战” --------------------------- 今明两天（5月19至20日），我国首轮大范围高温天气将达到鼎盛，多地将现37℃到39℃的高温，公众请注意防暑。 5月18日 北方首场高温天气将进入鼎盛阶段 江南华南多地需警惕降雨叠加致灾 ------------------------------- 今天北方首场高温过程已开启，明（5月19日）起至22日将进入核心影响时段，多地或经历40℃的高温酷热天气。南方未来一周仍维持多雨模式。 5月17日 南方多轮降水接踵而至 下周初北方高温发展河南陕西局地或达40℃ ------------------------------- 南方未来三天（5月17日至19日）降雨连连，20日至21日、22至24日还将有降雨过程。气温方面，19日至21日我国高温明显增多、增强。 5月16日 南方大部陷入降雨车轮战 北方高温将增多增强 --------------------- 今天（5月16日），西南地区

In [4]:
# Using Language Models  使用语言模型
# 接下来，让我们学习如何使用语言模型来调用工具。LangChain 支持多种不同的语言模型，您可以互换使用 - 请在下方选择您想要使用的模型！

# Select chat model:
# 选择聊天模型 ：
# Google Gemini ▾
# pip install -qU "langchain[google-genai]"


In [5]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [6]:
# 您可以通过传入消息列表来调用语言模型。默认情况下，响应是 content 字符串。


In [7]:
query = "你好!"
response = model.invoke([{"role": "user", "content": query}])
response.text()


'你好！很高兴能和你交流。有什么我可以帮你的吗？'

In [8]:
# 现在我们可以看看如何让这个模型进行工具调用。为了实现这一点，我们使用 .bind_tools 来为语言模型提供这些工具的知识。


In [9]:
model_with_tools = model.bind_tools(tools)

In [10]:
# 现在我们可以调用该模型了。我们先用普通消息调用它，看看它是如何响应的。我们可以查看 content 字段和 tool_calls 字段。

In [11]:
query = "你好!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 你好！很高兴为您服务。有什么我可以帮您的吗？

Tool calls: []


In [12]:
# 现在，让我们尝试使用一些需要调用工具的输入来调用它。


In [13]:
query = "搜索北京的天气"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': '北京的天气'}, 'id': '416a996c-51dc-421e-b37a-1754e714c74a', 'type': 'tool_call'}]


In [14]:
# 我们可以看到，现在没有文本内容，但是有一个工具调用！它希望我们调用 Tavily Search 工具。
# 这还不算调用那个工具——它只是告诉我们调用它。为了真正调用它，我们需要创建代理。


In [15]:
# Create the agent  创建代理
# 现在我们已经定义了工具和 LLM，可以创建代理了。我们将使用 LangGraph 来构建代理。目前，我们使用一个高级接口来构建代理，但 LangGraph 的优点在于，这个高级接口背后有一个低级、高度可控的 API，方便您修改代理逻辑。
# 现在，我们可以使用 LLM 和工具初始化代理。
# 注意，我们传入的是 model ，而不是 model_with_tools 。这是因为 create_react_agent 会在后台为我们调用 .bind_tools 。


In [16]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [17]:
# Run the agent  运行代理
# 现在，我们可以通过几个查询来运行代理！请注意，目前这些都是无状态查询（它不会记住之前的交互）。需要注意的是，代理将在交互结束时返回最终状态（其中包含所有输入，稍后我们将了解如何仅获取输出）。
# 首先，让我们看看当不需要调用工具时它如何响应：


In [18]:
input_message = {"role": "user", "content": "你好!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

你好!
================================== Ai Message ==================================

你好！有什么我可以帮你的吗？


In [19]:
# 为了确切了解幕后发生的事情（并确保它没有调用工具），我们可以查看 LangSmith 跟踪


In [20]:
# 现在让我们在一个应该调用该工具的示例上尝试一下


In [22]:
input_message = {"role": "user", "content": "搜索北京的天气"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

搜索北京的天气
================================== Ai Message ==================================
Tool Calls:
  tavily_search (a692c29a-4062-4900-a2f5-ce382013af91)
 Call ID: a692c29a-4062-4900-a2f5-ce382013af91
  Args:
    query: 北京的天气
================================= Tool Message =================================
Name: tavily_search

{"query": "北京的天气", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.weather.com.cn/weather/101010100.shtml", "title": "预报- 北京 - 中国天气网", "content": "<<返回 全国 全国>北京>城区 31/18℃ <3级 晴转多云 33/20℃ <3级 29/18℃ <3级 26/17℃ <3级 23/16℃ <3级 26/18℃ <3级 27/19℃ <3级 适宜 洗车指数 适宜 洗车指数 较适宜 洗车指数 适宜 洗车指数 适宜 洗车指数 适宜 洗车指数 过去24小时最大风力: 3级 (h)℃(μg/m³)(%)(mm)(级) 40°C 35°C 30°C 25°C 20°C 15°C 10°C 31  18℃  33  20℃  29  18℃  26  17℃  23  16℃  26  18℃  27  19℃  周边地区 | 周边景点 2025-05-19 07:30更新 周边地区 | 周边景点 2025-05-19 07:30更新 5月19日 今年首场大范围高温进入鼎盛阶段 南方降雨陷入“车轮战” ------------------

In [23]:
# 我们可以检查 LangSmith 跟踪以确保它有效地调用搜索工具。


In [24]:
# Streaming Messages
# 我们已经了解了如何使用 .invoke 调用代理来获取最终响应。如果代理执行多个步骤，这可能需要一段时间。为了显示中间进度，我们可以在消息发生时进行流式传输。




In [25]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

搜索北京的天气
================================== Ai Message ==================================
Tool Calls:
  tavily_search (6874f7e1-ce07-47b6-835b-8cd901b02fcf)
 Call ID: 6874f7e1-ce07-47b6-835b-8cd901b02fcf
  Args:
    query: 北京的天气
================================= Tool Message =================================
Name: tavily_search

{"query": "北京的天气", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.weather.com.cn/weather/101010100.shtml", "title": "预报- 北京 - 中国天气网", "content": "<<返回 全国 全国>北京>城区 31/18℃ <3级 晴转多云 33/20℃ <3级 29/18℃ <3级 26/17℃ <3级 23/16℃ <3级 26/18℃ <3级 27/19℃ <3级 适宜 洗车指数 适宜 洗车指数 较适宜 洗车指数 适宜 洗车指数 适宜 洗车指数 适宜 洗车指数 过去24小时最大风力: 3级 (h)℃(μg/m³)(%)(mm)(级) 40°C 35°C 30°C 25°C 20°C 15°C 10°C 31  18℃  33  20℃  29  18℃  26  17℃  23  16℃  26  18℃  27  19℃  周边地区 | 周边景点 2025-05-19 07:30更新 周边地区 | 周边景点 2025-05-19 07:30更新 5月19日 今年首场大范围高温进入鼎盛阶段 南方降雨陷入“车轮战” ------------------

In [26]:
# Streaming tokens
# 除了流式返回消息之外，流式返回令牌也很有用。我们可以通过指定 stream_mode="messages" 来实现。
# 下面我们使用 message.text() ，它需要 langchain-core>=0.3.37 。



In [27]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

北京|城区今天31/18℃，晴转多云，<3级。|未来几天天气预报：明天33/20℃，<3级；后|天29/18℃，<3级。|

In [28]:
# Adding in memory  添加内存
# 如前所述，此代理是无状态的。这意味着它不记得之前的交互。为了赋予它记忆，我们需要传入一个检查点。传入检查点时，我们还必须在调用代理时传入一个 thread_id （以便它知道从哪个线程/对话恢复）。



In [29]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [30]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [31]:
input_message = {"role": "user", "content": "你好，我是初音未来!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

你好，我是初音未来!
================================== Ai Message ==================================

こんにちは、初音ミクさん！お会いできて嬉しいです！何かお手伝いできることはありますか？


In [32]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is 初音ミク (Hatsune Miku).


In [33]:
# 如果你想开始一个新的对话，你所要做的就是改变使用的 thread_id

In [34]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I do not have access to personal information. I cannot know your name.


In [35]:
# Conclusion  结论
# 好了！在本快速入门中，我们介绍了如何创建一个简单的代理。然后，我们演示了如何流式传输响应——不仅包含中间步骤，还包含令牌！我们还添加了内存，以便您可以与代理进行对话。代理是一个复杂的主题，有很多东西需要学习！
